In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import os
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import v2
from torchvision.transforms.functional import adjust_contrast
import matplotlib.pyplot as plt
import glob

In [21]:
from dataset import *

In [22]:
transform = transforms.Compose([
    transforms.ToTensor(),                # from [0,255] to range [0.0,1.0]
    v2.RandomCrop(size=(256, 256))
    ])

input_images = glob.glob('data/training/INPUT_IMAGES/*P1.5.JPG')
train_dataset = CorrectionImageDataset(input_images, transform=transform, return_og=True)
validation_dataset = CorrectionImageDataset('data/validation/GT_IMAGES', transform=transform)

# test set must be done with the same data 
new_im, og_im = train_dataset[5]

fig, axs = plt.subplots(1,2)
axs[0].imshow(new_im.permute((1,2,0)))
axs[1].imshow(og_im.permute((1,2,0)))
print((new_im-og_im).sum())

error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [18]:
len(train_dataset)

3535

In [ ]:
len(validation_dataset)

# Training the model

In [ ]:
from ddpm.ddpm import *
from ddpm.ddpm_train import *

BATCH_SIZE=2
RESIZE_SIZE = 256
IM_SIZE=64


transform = transforms.Compose([
    transforms.ToTensor(),                # from [0,255] to range [0.0,1.0]
    v2.Resize(size=(RESIZE_SIZE, RESIZE_SIZE)),
    v2.RandomCrop(size=(IM_SIZE, IM_SIZE))
    ])

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else None)  
print(f"Model will run on {device}")


train_dataset = CorrectionImageDataset('data/training/GT_IMAGES', transform=transform, return_og=False)
dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train(dataloader, device=device, T=250, img_size=IM_SIZE, batch_size=BATCH_SIZE)